<a href="https://colab.research.google.com/github/arielwendichansky/DI_Bootcamp/blob/master/Week8/Day3/Sequence_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install tensorflow
! pip install keras

In [5]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical

In [6]:
# Sample data: Sequences of numbers
words = [
    'machine', 'learning', 'artificial', 'intelligence', 'neural',
    'network', 'algorithm', 'computer', 'programming', 'science',
    'data', 'analysis', 'modeling', 'statistics', 'engineering',
    'technology', 'digital', 'research', 'development', 'automation'
]

# Initialize the Tokenizer
tokenizer = Tokenizer(char_level=True)  # char_level=True to tokenize at the character level
tokenizer.fit_on_texts(words)

In [9]:
# Convert words to sequences of integers (vectorization)
sequences = tokenizer.texts_to_sequences(words)

# Padding sequences to ensure uniform length
max_len = max([len(seq) for seq in sequences]) - 1  # Max length of sequences
X = [seq[:-1] for seq in sequences]
y = [seq[-1] for seq in sequences]

X = pad_sequences(X, maxlen=max_len, padding='post')
y = np.array(y)

# One-hot encode the output variable
y = to_categorical(y, num_classes=len(tokenizer.word_index) + 1)  # +1 for zero indexing

# Building the RNN Model
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(max_len, 1)))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))  # Adjust output layer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Train the model
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
1/1 [==============================] - 0s 18ms/step - loss: 0.0551 - accuracy: 1.0000
Epoch 2/50
1/1 [==============================] - 0s 15ms/step - loss: 0.0498 - accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 16ms/step - loss: 0.0452 - accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 17ms/step - loss: 0.0410 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 15ms/step - loss: 0.0371 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 15ms/step - loss: 0.0336 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 15ms/step - loss: 0.0304 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 15ms/step - loss: 0.0277 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 15ms/step - loss: 0.0254 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 16ms/step - loss: 0.0232 - accuracy: 1.0000
Epoch 11/50
1/1 [============

In [16]:
def predict_last_char(word):
    sequence = tokenizer.texts_to_sequences([word])[0]
    sequence = pad_sequences([sequence], maxlen=max_len, padding='post')
    prediction = model.predict(sequence, verbose=0)
    predicted_index = np.argmax(prediction)
    return tokenizer.sequences_to_texts([[predicted_index]])[0]

# Test the model
test_word = 'developi'
predicted_char = predict_last_char(test_word)
print(f"Predicted last character for '{test_word}': {predicted_char}")

Predicted last character for 'developi': h


# LSTM And GRU for RNN

In [17]:
import numpy as np

def forget_layer(combine):
    # Dummy implementation; in practice, this is a sigmoid activation
    return np.sigmoid(combine)

def candidate_layer(combine):
    # Dummy implementation; in practice, this is a tanh activation
    return np.tanh(combine)

def input_layer(combine):
    # Dummy implementation; in practice, this is a sigmoid activation
    return np.sigmoid(combine)

def output_layer(combine):
    # Dummy implementation; in practice, this is a sigmoid activation
    return np.sigmoid(combine)

def LSTMCELL(prev_ct, prev_ht, input):
    combine = prev_ht + input
    ft = forget_layer(combine)
    candidate = candidate_layer(combine)
    it = input_layer(combine)
    Ct = prev_ct * ft + candidate * it
    ot = output_layer(combine)
    ht = ot * np.tanh(Ct)

    return ht, Ct

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Sample text data
sentences = [
    "I am from Brazil, so at school I had to learn how to speak Portuguese",
    "She is from Japan, so at school she had to learn how to speak Japanese",
    "He is from Spain, so at school he had to learn how to speak Spanish",
    "As a Canadian, in my school, French was a compulsory language to learn",
    "Living in India exposes you to many languages, but Hindi is often a necessity",
    "In the USA, many students choose to learn Spanish as a second language",
    "In Russia, understanding Russian is essential, especially in the education system",
    "In China, Mandarin is the primary language taught in schools",
    "Italy is known for its beautiful language, so Italian is key in their schools",
    "In Germany, learning German is a fundamental part of their education system",
    "Being in Egypt, it is common to learn Arabic in schools",
]

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1

# Create input sequences and labels
input_sequences = []
for line in sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Predictors and label
X, labels = input_sequences[:,:-1],input_sequences[:,-1]
y = tf.keras.utils.to_categorical(labels, num_classes=total_words)

# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#fit the model
model.fit(X, y, epochs=100, verbose=1)

#predict
def predict_next_word(text):
    sequence = tokenizer.texts_to_sequences([text])[0]
    sequence = pad_sequences([sequence], maxlen=max_sequence_len-1, padding='pre')
    prediction = model.predict(sequence, verbose=0)
    index = np.argmax(prediction)
    return tokenizer.index_word[index]

#test
input_text = "I am from Brazil, so at school I had to learn how to speak..."
predicted_word = predict_next_word(input_text)
print("Input:", input_text, "\nPredicted next word:", predicted_word)

Epoch 1/100
5/5 [==============================] - 2s 23ms/step - loss: 4.3284 - accuracy: 0.0227
Epoch 2/100
5/5 [==============================] - 0s 22ms/step - loss: 4.2959 - accuracy: 0.0833
Epoch 3/100
5/5 [==============================] - 0s 22ms/step - loss: 4.2386 - accuracy: 0.0909
Epoch 4/100
5/5 [==============================] - 0s 23ms/step - loss: 4.1052 - accuracy: 0.0758
Epoch 5/100
5/5 [==============================] - 0s 29ms/step - loss: 4.0382 - accuracy: 0.0985
Epoch 6/100
5/5 [==============================] - 0s 25ms/step - loss: 4.0070 - accuracy: 0.1136
Epoch 7/100
5/5 [==============================] - 0s 27ms/step - loss: 3.9585 - accuracy: 0.1136
Epoch 8/100
5/5 [==============================] - 0s 24ms/step - loss: 3.9455 - accuracy: 0.1061
Epoch 9/100
5/5 [==============================] - 0s 28ms/step - loss: 3.9182 - accuracy: 0.1061
Epoch 10/100
5/5 [==============================] - 0s 27ms/step - loss: 3.8729 - accuracy: 0.1061
Epoch 11/100
5/5 [=